In [57]:
import pandas as pd

In [58]:
from fbprophet import Prophet

In [59]:
from fbprophet.plot import plot_plotly

In [60]:
import plotly.offline as py

### Data set
* https://github.com/CSSEGISandData/COVID-19

Get the latest confirmed cases csv (Requires wget)

In [61]:
!wget -P ds/ --progress=bar -N https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv

--2020-07-16 21:09:56--  https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ds/time_series_covid19_confirmed_global.csv’ not modified on server. Omitting download.



In [62]:
odf = pd.read_csv('ds/time_series_covid19_confirmed_global.csv', header=None)
odf.head()

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
1,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,33190,33384,33594,33908,34194,34366,34451,34455,34740,34994
2,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2964,3038,3106,3188,3278,3371,3454,3571,3667,3752
3,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,16404,16879,17348,17808,18242,18712,19195,19689,20216,20770
4,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,855,855,855,855,855,855,855,858,861,862


## Which country?

In [63]:
# country = 'United Arab Emirates'

In [64]:
country = 'India'

### Create timeseries dataframe

### Time series data
Column a: Label: ds Data: Date
Column b: Label: y Data: Total count for the day

In [65]:
df = odf

In [66]:
c_idx = df.index[df[1] == country].to_list()

In [67]:
df.drop(df.index[1:c_idx[0]], inplace=True)

Drop the remaining countries after the selected country

In [68]:
df.drop(df.index[2:len(df)], inplace=True)

In [69]:
df

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
132,NaN,India,20.593684,78.96288,0,0,0,0,0,0,...,719664,742417,767296,793802,820916,849522,878254,906752,936181,968857


In [70]:
df.drop(df.columns[0:4], axis=1, inplace=True)

In [71]:
df

,4,5,6,7,8,9,10,11,12,13,...,170,171,172,173,174,175,176,177,178,179
0,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
132,0,0,0,0,0,0,0,0,1,1,...,719664,742417,767296,793802,820916,849522,878254,906752,936181,968857


In [72]:
df[1:].apply(pd.to_numeric)

,4,5,6,7,8,9,10,11,12,13,...,170,171,172,173,174,175,176,177,178,179
132,0,0,0,0,0,0,0,0,1,1,...,719664,742417,767296,793802,820916,849522,878254,906752,936181,968857


In [73]:
df = df.T

In [74]:
df

,0,132
4,1/22/20,0
5,1/23/20,0
6,1/24/20,0
7,1/25/20,0
8,1/26/20,0
...,...,...
175,7/11/20,849522
176,7/12/20,878254
177,7/13/20,906752
178,7/14/20,936181


In [75]:
df.index = [x for x in range(len(df))]

In [76]:
df

,0,132
0,1/22/20,0
1,1/23/20,0
2,1/24/20,0
3,1/25/20,0
4,1/26/20,0
...,...,...
171,7/11/20,849522
172,7/12/20,878254
173,7/13/20,906752
174,7/14/20,936181


In [77]:
df.columns = ['ds', 'y']

In [78]:
df.head()

,ds,y
0,1/22/20,0
1,1/23/20,0
2,1/24/20,0
3,1/25/20,0
4,1/26/20,0


In [79]:
len(df)

176

### Create model and fit

In [80]:
minus_5_days = yesterday = (pd.Timestamp.now() - pd.Timedelta(5, 'd')).strftime("%Y-%m-%d")

In [81]:
changepoints = ['2020-03-23', '2020-03-28', '2020-04-16', '2020-04-23', '2020-05-15', '2020-06-01', '2020-06-07', '2020-06-15', '2020-06-30', '2020-07-05']

In [82]:
changepoints.append(minus_5_days)

In [83]:
changepoints

['2020-03-23',
 '2020-03-28',
 '2020-04-16',
 '2020-04-23',
 '2020-05-15',
 '2020-06-01',
 '2020-06-07',
 '2020-06-15',
 '2020-06-30',
 '2020-07-05',
 '2020-07-11']

In [84]:
m = Prophet(weekly_seasonality=True,
            changepoint_prior_scale=.5,
            changepoints=changepoints) 

m.fit(df)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [85]:
#future = m.make_future_dataframe(periods=120)
future = m.make_future_dataframe(periods=90)
future.head()

,ds
0,2020-01-22
1,2020-01-23
2,2020-01-24
3,2020-01-25
4,2020-01-26


In [86]:
future.tail()

,ds
261,2020-10-09
262,2020-10-10
263,2020-10-11
264,2020-10-12
265,2020-10-13


In [ ]:
forecast = m.predict(future)

In [ ]:
forecast.head()

In [ ]:
m.plot(forecast);

In [ ]:
m.plot_components(forecast)

In [ ]:
py.init_notebook_mode()

In [ ]:
fig = plot_plotly(m, forecast, changepoints=True)

In [ ]:
py.iplot(fig)

### Validate results

In [ ]:
f_validate = forecast.copy()

In [ ]:
f_validate

#### Add actual data column

In [ ]:
f_validate['actual'] = df['y']

In [ ]:
f_validate.columns

In [ ]:
len(df)

In [ ]:
len(f_validate)

In [ ]:
f_validate

In [ ]:
f_validate['diff'] = f_validate['actual'].astype(float) - f_validate['yhat']

In [ ]:
f_validate['error'] = f_validate['diff'].astype(float) / f_validate['actual'].astype(float) * 100

In [ ]:
f_validate['accuracy'] = 100 - abs(f_validate['error'].astype(float))

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

#### Get last x number of days 

In [ ]:
time_range_days = 30

In [ ]:
yesterday = (pd.Timestamp.now() - pd.Timedelta(1, 'd')).strftime("%Y-%m-%d")

In [ ]:
s_date = pd.to_datetime((pd.Timestamp.now() - pd.Timedelta(time_range_days, 'd')).strftime("%Y-%m-%d"))
e_date = pd.to_datetime(yesterday)

In [ ]:
f_validate = f_validate.loc[(f_validate['ds'] >= s_date) & (f_validate['ds'] <= e_date)]

In [ ]:
f_validate[['ds', 'yhat', 'actual', 'diff', 'error', 'accuracy']]

In [ ]:
f_val_results = f_validate[['ds', 'yhat', 'actual', 'diff', 'error', 'accuracy']]


#### -ve values indicate model is predicting more than actual value

In [ ]:
f_val_results.plot(x='ds', y=['error', 'accuracy'], 
                        kind='area', stacked=False, 
                        colormap='RdYlBu', 
                        title='Error and Accuracy %', 
                        grid=True)

### Difference Between Actual & Forecast

If the graph is increasing then we have more cases than forecast.

In [ ]:
f_val_results.plot(x='ds', y=['diff'], 
                        kind='line', title="Difference Between Actual & Forecast")

In [ ]:
f_val_results.plot(x='ds', y=['error'], 
                        kind='line', title="Error % over time")